# Individual and gender inequality in computer science: A career study of cohorts from 1970 to 2000

## Part 2: Exploration

In this notebook, we explore patterns in the field of computer science and produce four figures. Figure 1 of the paper is a basic description of the field. Figure 2 shows individual inequality in productivity and impact as a function of career ages. Figure 3 shows individual inequality in productivity and impact as a function of cohorts. Figure 4 depicts gender inequality for productivity and impact as a function of cohorts and career ages.

---

### 1. Imports

Many of the custom functions we need are stored in a utilities file.

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import powerlaw as pl
import scipy as sp

from matplotlib.colorbar import ColorbarBase
from matplotlib.colors import Normalize
from matplotlib.ticker import MultipleLocator, FixedLocator
from utils import *

### 2. Directories

Create directories if they don't exist:

In [ ]:
directory = '../results'
if not os.path.exists(directory):
    os.makedirs(directory)

directory = '../results/fig1'
if not os.path.exists(directory):
    os.makedirs(directory)

directory = '../results/fig2'
if not os.path.exists(directory):
    os.makedirs(directory)

directory = '../results/fig3'
if not os.path.exists(directory):
    os.makedirs(directory)

directory = '../results/fig4'
if not os.path.exists(directory):
    os.makedirs(directory)

### 3. Load data

Load files from the 'data' directory:

In [ ]:
authors_publications = pd.read_csv('../data/authors_publications.csv.gz')
features = pd.read_csv('../data/features.csv.gz') 
counts = pd.read_csv('../data/counts.csv.gz')
counts_first = pd.read_csv('../data/counts_first.csv.gz')

Enrich the `counts` dataframes:

In [ ]:
dropout_cols = ['author', 'dropout']
counts = counts.merge(features[dropout_cols], on='author', how='left')
counts_first = counts_first.merge(features[dropout_cols], on='author', how='left')

### 4. Produce figures

#### 4.1. Figure 1

(A) The size of cohorts increases exponentially with time for both males and females:

In [ ]:
cohort_sizes = counts.groupby('cohort').agg({'author': 'nunique'})
cohort_sizes_m = counts[counts['gender'] == 'm'].groupby(['cohort']).agg({'author': 'nunique'})
cohort_sizes_f = counts[counts['gender'] == 'f'].groupby(['cohort']).agg({'author': 'nunique'})

In [ ]:
linewidth = 2
fontsize = 18
fig = plt.figure(figsize=(4, 4))
ax = fig.add_subplot(111)
ax.plot(cohort_sizes.index, cohort_sizes.values, linewidth=linewidth, ls=':', color='black', label='Total')
ax.plot(cohort_sizes_m.index, cohort_sizes_m.values, linewidth=linewidth, ls='--', color='blue', label='Male')
ax.plot(cohort_sizes_f.index, cohort_sizes_f.values, linewidth=linewidth, ls='-', color='red', label='Female')
ax.xaxis.set_ticks_position('both')
ax.yaxis.set_ticks_position('both')
ax.set_yscale('log')
ax.set_xlabel('Cohort', fontsize=fontsize)
ax.set_ylabel('Number of Authors', fontsize=fontsize)
ax.set_xticks([1970, 1980, 1990, 2000])
ax.tick_params(axis='x', which='major', direction='in', width=linewidth, size=4*linewidth, labelsize=fontsize)
ax.tick_params(axis='x', which='minor', direction='in', width=linewidth, size=2*linewidth, labelsize=fontsize)
ax.tick_params(axis='y', which='major', direction='in', width=linewidth, size=4*linewidth, labelsize=fontsize)
ax.tick_params(axis='y', which='minor', direction='in', width=linewidth, size=2*linewidth, labelsize=fontsize)
ax.spines['left'].set_linewidth(linewidth)
ax.spines['right'].set_linewidth(linewidth)
ax.spines['bottom'].set_linewidth(linewidth)
ax.spines['top'].set_linewidth(linewidth)
ax.legend(fontsize=fontsize-6)
plt.gcf().text(0., 0.9, 'A', fontsize=fontsize*2)
plt.subplots_adjust(left=0.25, right=0.95, bottom=0.2, top=0.9)
fig.savefig('../results/fig1/fig1a_authors_per_cohort.pdf')

(B) The average team size, measured by the number of authors per paper, increases over time:

In [ ]:
avg_team_size = authors_publications.groupby(['year', 'pub_id']).size().groupby('year').mean().to_frame('avg_team_size')
avg_team_size = avg_team_size[avg_team_size.index.isin(list(range(1970, 2015)))]

In [ ]:
linewidth = 2
fontsize = 18
fig = plt.figure(figsize=(4, 4))
ax = fig.add_subplot(111)
ax.plot(avg_team_size.index, avg_team_size.avg_team_size, linewidth=linewidth, color='black')
ax.xaxis.set_ticks_position('both')
ax.yaxis.set_ticks_position('both')
ax.set_xlabel('Year', fontsize=fontsize)
ax.set_ylabel('Average Team Size', fontsize=fontsize)
ax.set_xticks([1970, 1990, 2010])
ax.tick_params(axis='x', which='major', direction='in', width=linewidth, size=4*linewidth, labelsize=fontsize)
ax.tick_params(axis='x', which='minor', direction='in', width=linewidth, size=2*linewidth, labelsize=fontsize)
ax.tick_params(axis='y', which='major', direction='in', width=linewidth, size=4*linewidth, labelsize=fontsize)
ax.tick_params(axis='y', which='minor', direction='in', width=linewidth, size=2*linewidth, labelsize=fontsize)
ax.spines['left'].set_linewidth(linewidth)
ax.spines['right'].set_linewidth(linewidth)
ax.spines['bottom'].set_linewidth(linewidth)
ax.spines['top'].set_linewidth(linewidth)
plt.gcf().text(0., 0.9, 'B', fontsize=fontsize*2)
plt.subplots_adjust(left=0.25, right=0.95, bottom=0.2, top=0.9)
fig.savefig('../results/fig1/fig1b_average_team_size_per_year.pdf')

(C) Distributions of productivity and impact are broad:

In [ ]:
# extract data
P_15 = counts[counts['career_age'] == 15]['cum_num_pub'].to_list()
P_15 = [int(x) for x in P_15 if x>0]
C_15 = counts[counts['career_age'] == 15]['cum_num_cit'].to_list()
C_15 = [int(x) for x in C_15 if x>0]

# bin data
a_bin_P_15 = bin_pdf(pdf(P_15))
a_bin_C_15 = bin_pdf(pdf(C_15))

# fit data
f_P_15 = pl.Fit(P_15, discrete=True, xmin=1)
f_C_15 = pl.Fit(C_15, discrete=True, xmin=1)

# create discrete x-coordinates 
space_xmin_P_15 = np.logspace(np.log10(f_P_15.xmin), np.log10(max(f_P_15.data_original)), 100)
space_xmin_C_15 = np.logspace(np.log10(f_C_15.xmin), np.log10(max(f_C_15.data_original)), 100)

In [ ]:
fontsize = 18
linewidth = 2
markersize = 9
fig = plt.figure(figsize=(4, 4))
ax = fig.add_subplot(111)
ax.plot(a_bin_P_15[:, 0], a_bin_P_15[:, 3], marker='^', color='green', ls='', markersize=markersize, label='$x=P(15)$')
ax.plot(a_bin_C_15[:, 0], a_bin_C_15[:, 3], marker='v', color='purple', ls='', markersize=markersize, label='$x=C(15)$')
ax.plot(space_xmin_P_15, f_P_15.truncated_power_law.pdf(space_xmin_P_15), color='k', ls='-', linewidth=linewidth)
ax.plot(space_xmin_C_15, f_C_15.lognormal.pdf(space_xmin_C_15), color='k', ls='--', linewidth=linewidth)
ax.xaxis.set_ticks_position('both')
ax.yaxis.set_ticks_position('both')
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('$x$', fontsize=fontsize)
ax.set_ylabel('Probability', fontsize=fontsize)
ax.set_xticks([1, 10, 100, 1000, 10000])
ax.tick_params(axis='x', which='major', direction='in', width=linewidth, size=4*linewidth, labelsize=fontsize)
ax.tick_params(axis='x', which='minor', direction='in', width=linewidth, size=2*linewidth, labelsize=fontsize)
ax.tick_params(axis='y', which='major', direction='in', width=linewidth, size=4*linewidth, labelsize=fontsize)
ax.tick_params(axis='y', which='minor', direction='in', width=linewidth, size=2*linewidth, labelsize=fontsize)
ax.spines['left'].set_linewidth(linewidth)
ax.spines['right'].set_linewidth(linewidth)
ax.spines['bottom'].set_linewidth(linewidth)
ax.spines['top'].set_linewidth(linewidth)
ax.legend(fontsize=fontsize-6)
plt.gcf().text(0., 0.9, 'C', fontsize=fontsize*2)
plt.subplots_adjust(left=0.25, right=0.95, bottom=0.2, top=0.9)
fig.savefig('../results/fig1/fig1c_distributions.pdf')

(D) The number of authors decreases with the number of years during which they publish persistently after the beginning of their careers:

In [ ]:
authors = features[['author', 'cohort', 'gender', 'last_consec_ca', 'max_absence', 'career_length']]

persistence = authors['last_consec_ca'].value_counts().sort_index()
persistence_female = authors[authors['gender'] == 'f']['last_consec_ca'].value_counts().sort_index()
persistence_female = np.array([persistence_female.index, persistence_female.values, persistence_female.values/sum(persistence_female.values)]).T
persistence_male = authors[authors['gender'] == 'm']['last_consec_ca'].value_counts().sort_index()
persistence_male = np.array([persistence_male.index, persistence_male.values, persistence_male.values/sum(persistence_male.values)]).T

In [ ]:
linewidth = 2
fontsize = 18
markersize = 9
fig = plt.figure(figsize=(4, 4))
ax = fig.add_subplot(111)
ax.plot(persistence_female[:14, 0], persistence_female[:14, 2], color='red', linewidth=2, ls='-', label='Female')
ax.plot(persistence_male[:14, 0], persistence_male[:14, 2], color='blue', linewidth=2, ls='--', label='Male')
ax.set_yscale('log')
ax.set_ylabel('Probability', fontsize=fontsize)
ax.set_xticks([1, 5, 10, 14])
ax.set_xlabel('Early Career Persistence', fontsize=fontsize)
ax.tick_params(axis='x', which='major', direction='in', width=linewidth, size=4*linewidth, labelsize=fontsize)
ax.tick_params(axis='x', which='minor', direction='in', width=linewidth, size=2*linewidth, labelsize=fontsize)
ax.tick_params(axis='y', which='major', direction='in', width=linewidth, size=4*linewidth, labelsize=fontsize)
ax.tick_params(axis='y', which='minor', direction='in', width=linewidth, size=2*linewidth, labelsize=fontsize)
ax.spines['left'].set_linewidth(linewidth)
ax.spines['right'].set_linewidth(linewidth)
ax.spines['bottom'].set_linewidth(linewidth)
ax.spines['top'].set_linewidth(linewidth)
plt.legend(fontsize=fontsize-6)
plt.gcf().text(0., 0.9, 'D', fontsize=fontsize*2)
plt.subplots_adjust(left=0.25, right=0.95, bottom=0.2, top=0.9)
fig.savefig('../results/fig1/fig1d_persistence_gender.pdf')

(E) The fraction of authors in a cohort that drop out of academia decreases but is more or less constant since the mid-80s:

In [ ]:
cohort_sizes_dropout = counts[counts['dropout'] == True].groupby(['cohort']).agg({'author': 'nunique'})
cohort_sizes_m_dropout = counts[(counts['gender'] == 'm') & (counts['dropout'] == True)].groupby(['cohort']).agg({'author': 'nunique'})
cohort_sizes_f_dropout = counts[(counts['gender'] == 'f') & (counts['dropout'] == True)].groupby(['cohort']).agg({'author': 'nunique'})

cohort_sizes_dropout_rate = cohort_sizes_dropout/cohort_sizes
cohort_sizes_m_dropout_rate = cohort_sizes_m_dropout/cohort_sizes_m
cohort_sizes_f_dropout_rate = cohort_sizes_f_dropout/cohort_sizes_f

In [ ]:
linewidth = 2
fontsize = 18
fig = plt.figure(figsize=(4, 4))
ax = fig.add_subplot(111)
ax.plot(cohort_sizes_dropout_rate.index, cohort_sizes_dropout_rate.values, linewidth=linewidth, ls=':', color='black', label='Total')
ax.plot(cohort_sizes_m_dropout_rate.index, cohort_sizes_m_dropout_rate.values, linewidth=linewidth, ls='--', color='blue', label='Male')
ax.plot(cohort_sizes_f_dropout_rate.index, cohort_sizes_f_dropout_rate.values, linewidth=linewidth, ls='-', color='red', label='Female')
ax.xaxis.set_ticks_position('both')
ax.yaxis.set_ticks_position('both')
ax.set_xlabel('Cohort', fontsize=fontsize)
ax.set_ylabel('Dropout Rate', fontsize=fontsize)
ax.set_xticks([1970, 1980, 1990, 2000])
ax.tick_params(axis='x', which='major', direction='in', width=linewidth, size=4*linewidth, labelsize=fontsize)
ax.tick_params(axis='x', which='minor', direction='in', width=linewidth, size=2*linewidth, labelsize=fontsize)
ax.tick_params(axis='y', which='major', direction='in', width=linewidth, size=4*linewidth, labelsize=fontsize)
ax.tick_params(axis='y', which='minor', direction='in', width=linewidth, size=2*linewidth, labelsize=fontsize)
ax.spines['left'].set_linewidth(linewidth)
ax.spines['right'].set_linewidth(linewidth)
ax.spines['bottom'].set_linewidth(linewidth)
ax.spines['top'].set_linewidth(linewidth)
ax.legend(fontsize=fontsize-6)
plt.gcf().text(0., 0.9, 'E', fontsize=fontsize*2)
plt.subplots_adjust(left=0.25, right=0.95, bottom=0.2, top=0.9)
fig.savefig('../results/fig1/fig1e_dropout_rate_per_cohort.pdf')

#### 4.2. Figures 2 and 6

Individual inequality in productivity and impact as a function of career ages:

In [ ]:
def aggregate_career_age_data(counts, func, func_name):#, remove_zero=False):
    cohort_counts = counts.groupby(['cohort', 'career_age']).agg({'cum_num_pub':func, 'cum_num_cit':func, 'win_num_pub':func, 'win_num_cit':func})
    #if remove_zero:
    #    raise Exception('DONT REMOVE ZERO!')
    #    cohort_counts_pub_gt0 = counts[counts['win_num_pub']>0].groupby(['cohort', 'career_age']).agg({'win_num_cit':func, 'win_num_pub':func})
    #    cohort_counts['win_num_cit'] = cohort_counts_pub_gt0['win_num_cit']
    #    cohort_counts['win_num_pub'] = cohort_counts_pub_gt0['win_num_pub']
    cohort_counts.reset_index(inplace=True)
    cohort_counts = cohort_counts.rename({'cum_num_pub':f'{func_name}_cum_num_pub', 'cum_num_cit':f'{func_name}_cum_num_cit','win_num_pub':f'{func_name}_win_num_pub','win_num_cit': f'{func_name}_win_num_cit'}, axis='columns')
    return cohort_counts

def plot_criteria_over_career_ages(data, criteria, criteria_name, title, letter, x_start=1, x_end=15, legend=True, name_ext=''):
    linewidth = 2
    fontsize = 18
    fig = plt.figure(figsize=(4, 4))
    ax = fig.add_subplot(111)
    color = ['#000000', '#e41a1c', '#377eb8', '#4daf4a', '#984ea3', '#ff7f00', '#ffff33', '#a65628', '#f781bf', '#999999']
    cohort = [1970, 1975, 1980, 1985, 1990, 1995, 2000]
    for i in range(0, 7):
        df = data[data['cohort'] == cohort[i]]
        df = df[(df['career_age'] >= x_start) & (df['career_age'] <= x_end)]
        ax.plot(df['career_age'], df[criteria], linewidth=linewidth, label=cohort[i], color=color[i])
    ax.set_ylim([0.16, 1.04])
    ax.xaxis.set_ticks_position('bottom')
    ax.yaxis.set_ticks_position('both')
    ax.xaxis.set_minor_locator(MultipleLocator(1))
    ax.set_xlabel('Career Age', fontsize=fontsize)
    ax.set_ylabel(f'{criteria_name}', fontsize=fontsize)
    ax.set_title(title, fontsize=fontsize)
    ax.tick_params(axis='x', which='major', direction='in', width=linewidth, size=4*linewidth, labelsize=fontsize)
    ax.tick_params(axis='x', which='minor', direction='in', width=linewidth, size=2*linewidth, labelsize=fontsize)
    ax.tick_params(axis='y', which='major', direction='in', width=linewidth, size=4*linewidth, labelsize=fontsize)
    ax.tick_params(axis='y', which='minor', direction='in', width=linewidth, size=2*linewidth, labelsize=fontsize)
    ax.spines['left'].set_linewidth(linewidth)
    ax.spines['right'].set_linewidth(linewidth)
    ax.spines['bottom'].set_linewidth(linewidth)
    ax.spines['top'].set_linewidth(linewidth)
    if x_end == 13:
        ax.set_xlim([0.25, 13.75])
        ax.set_xticks([])
        ax.xaxis.set_major_locator(FixedLocator([1, 7, 13]))
        ax.set_xticks([1, 7, 13])
        ax.set_xticklabels(['3', '9', '15'])
    else:
        ax.set_xlim([0.25, 15.75])
        ax.set_xticks([1, 5, 10, 15])
    if legend: ax.legend(fontsize=fontsize-6)
    plt.gcf().text(0., 0.9, letter, fontsize=fontsize*2)
    plt.subplots_adjust(left=0.25, right=0.95, bottom=0.2, top=0.9)
    fig.savefig(f'../results/fig2/fig2_{criteria}{name_ext}.pdf')

def plot_array_configs(data, configs, letters, x_ends, name_ext=''):
    for config, letter, x_end in zip(configs, letters, x_ends):
        legend = False
        if letter == 'A': legend = True
        plot_criteria_over_career_ages(data, *config, letter=letter, legend=legend, x_end=x_end, name_ext=name_ext)

def get_config(crit, crit_name, size=''):
    config = [(f'{crit}_cum_num_pub', crit_name, f'Productivity {size}'), # cumulative
              (f'{crit}_win_num_pub', crit_name, f'Productivity {size}'), # window
              (f'{crit}_cum_num_cit', crit_name, f'Impact {size}'), # cumulative
              (f'{crit}_win_num_cit', crit_name, f'Impact {size}')] # window
    return config

In [ ]:
letters11 = ['A', 'A', 'B', 'B']
letters12 = ['C', 'C', 'D', 'D']
letters21 = ['E', 'E', 'F', 'F']
letters22 = ['G', 'G', 'H', 'H']
letters31 = ['I', 'I', 'J', 'J']
letters32 = ['K', 'K', 'L', 'L']

x_ends = [15, 13, 15, 13]

# all authors: every-author assignment
cohort_counts_gini = aggregate_career_age_data(counts, gini, 'gini')
plot_array_configs(cohort_counts_gini, get_config('gini', 'Gini'), letters11, x_ends)

# all authors: first-author assignment
cohort_counts_gini_first = aggregate_career_age_data(counts_first, gini, 'gini')
plot_array_configs(cohort_counts_gini_first, get_config('gini', 'Gini'), letters12, x_ends, name_ext='_first')

# dropouts removed: every-author assignment
counts_stayed = counts[counts.dropout == False]
cohort_counts_stayed_gini = aggregate_career_age_data(counts_stayed, gini, 'gini')
plot_array_configs(cohort_counts_stayed_gini, get_config('gini', 'Gini'), letters21, x_ends, name_ext='_stay10')

# dropouts removed: first-author assignment
counts_stayed_first = counts_first[counts_first.dropout == False]
cohort_counts_stayed_gini_first = aggregate_career_age_data(counts_stayed_first, gini, 'gini')
plot_array_configs(cohort_counts_stayed_gini_first, get_config('gini', 'Gini'), letters22, x_ends, name_ext='_stay10_first')

#### 4.3. Figures 3 and 7

Individual inequality in productivity and impact as a function of cohorts:

In [ ]:
def aggregate_cohort_data(citations_window, func):
    gini_cohorts_ca = citations_window.groupby(['cohort', 'career_age']).agg({'num_pub': func, 'num_cit': func, 'cum_num_pub': func, 'cum_num_cit': func}).reset_index()
    return gini_cohorts_ca

def plot_criteria_over_cohorts(data, criteria, criteria_name, title, letter, legend=True, name_ext=''):
    linewidth = 2
    fontsize = 18
    fig = plt.figure(figsize=(4, 4))
    ax = fig.add_subplot(111)
    color = ['#000000', '#e41a1c', '#377eb8', '#4daf4a', '#984ea3', '#ff7f00', '#ffff33', '#a65628', '#f781bf', '#999999']
    career_ages = [15, 10, 5, 3]
    for i in range(0, len(career_ages)):
        df = data[data['career_age'] == career_ages[i]]
        ax.plot(df['cohort'], df[criteria], linewidth=linewidth, label=career_ages[i], color=color[i])
    ax.set_ylim([0.16, 1.04])
    ax.xaxis.set_ticks_position('bottom')
    ax.yaxis.set_ticks_position('both')
    ax.set_xlabel('Cohort', fontsize=fontsize)
    ax.set_ylabel(f'{criteria_name}', fontsize=fontsize)
    ax.set_title(title, fontsize=fontsize)
    ax.set_xticks([1970, 1980, 1990, 2000])
    ax.tick_params(axis='x', which='major', direction='in', width=linewidth, size=4*linewidth, labelsize=fontsize)
    ax.tick_params(axis='x', which='minor', direction='in', width=linewidth, size=2*linewidth, labelsize=fontsize)
    ax.tick_params(axis='y', which='major', direction='in', width=linewidth, size=4*linewidth, labelsize=fontsize)
    ax.tick_params(axis='y', which='minor', direction='in', width=linewidth, size=2*linewidth, labelsize=fontsize)
    ax.spines['left'].set_linewidth(linewidth)
    ax.spines['right'].set_linewidth(linewidth)
    ax.spines['bottom'].set_linewidth(linewidth)
    ax.spines['top'].set_linewidth(linewidth)
    if legend: ax.legend(fontsize=fontsize-6)
    plt.gcf().text(0., 0.9, letter, fontsize=fontsize*2)
    plt.subplots_adjust(left=0.25, right=0.95, bottom=0.2, top=0.9)
    fig.savefig(f'../results/fig3/fig3_{criteria}_over_cohorts{name_ext}.pdf')

def plot_array_configs2(data, configs, letters, name_ext=''):
    for config, letter in zip(configs, letters):
        legend = False
        if letter in ['A', 'E', 'I']: legend = True
        plot_criteria_over_cohorts(data, *config, letter=letter, legend=legend, name_ext=name_ext)

def get_config2(crit, crit_name, size=''):
    config2 = [('cum_num_pub', crit_name, 'Productivity'), # cumulative
               ('num_pub', crit_name, 'Productivity'), # window
               ('cum_num_cit', crit_name, 'Impact'), # cumulative
               ('num_cit', crit_name, 'Impact')] # window
    return config2

In [ ]:
# all authors: every-author assignment
gini_cohorts_ca = aggregate_cohort_data(counts, gini)
plot_array_configs2(gini_cohorts_ca, get_config2('gini', 'Gini'), letters11)

# all authors: first-author assignment
gini_cohorts_ca_first = aggregate_cohort_data(counts_first, gini)
plot_array_configs2(gini_cohorts_ca_first, get_config2('gini', 'Gini'), letters12, name_ext='_first')

# dropouts removed: every-author assignment
counts_stayed = counts[counts.dropout == False]
gini_cohorts_ca_stayed = aggregate_cohort_data(counts_stayed, gini)
plot_array_configs2(gini_cohorts_ca_stayed, get_config2('gini', 'Gini'), letters21, name_ext='_stay10')

# dropouts removed: first-author assignment
counts_stayed_first = counts_first[counts_first.dropout == False]
gini_cohorts_ca_stayed_first = aggregate_cohort_data(counts_stayed_first, gini)
plot_array_configs2(gini_cohorts_ca_stayed_first, get_config2('gini', 'Gini'), letters22, name_ext='_stay10_first')

#### 4.4. Figure 4

Gender inequality for productivity and impact as a function of cohorts and career ages:

In [ ]:
def get_cohort_effect_size(cohort_careerage_df, metric, gen_a='m', gen_b='f', eff_form='r'):
    data = cohort_careerage_df[cohort_careerage_df.gender.isin([gen_a, gen_b])]
    data = data.set_index(['cohort', 'career_age', 'gender']).unstack(level=-1)
    data.columns = ['_'.join(col) for col in data.columns]
    data['cliffd_m_f'] = data.apply(lambda x: cliffsD(x[f'{metric}_{gen_a}'], x[f'{metric}_{gen_b}']), axis=1)
    mwu = data.apply(lambda x: mann_whitney_effect_size(x[f'{metric}_{gen_a}'], x[f'{metric}_{gen_b}'], effect_formula=eff_form), axis=1).apply(pd.Series)
    mwu.columns = ['effect', 'statistic', 'pvalue']
    data = data.join(mwu)
    data = data[['cliffd_m_f', 'effect', 'statistic', 'pvalue']]
    data = data.reset_index()
    return data

def horizonal_inequality(df, letter, title):
    df = df.copy()
    df.loc[df['pvalue'] > 0.05, 'cliffd_m_f'] = 0
    df = df.pivot(index='career_age', columns='cohort', values='cliffd_m_f').sort_index(ascending=False)
    linewidth = 2
    fontsize = 18
    fig = plt.figure(figsize=(4, 4))
    ax = fig.add_subplot(111)
    plot = ax.imshow(df, cmap='bwr', vmin=-0.17490821707496162, vmax=0.17490821707496162, aspect=31/15)
    ax.xaxis.set_ticks_position('both')
    ax.yaxis.set_ticks_position('both')
    ax.set_xticks([0, 10, 20, 30])
    ax.set_xticklabels(['1970', '1980', '1990', '2000'])
    ax.xaxis.set_minor_locator(MultipleLocator(1))
    ax.set_yticks([0, 5, 10, 14])
    ax.set_yticklabels(['15', '10', '5', '1'])
    ax.yaxis.set_minor_locator(MultipleLocator(1))
    ax.set_xlabel('Cohort', fontsize=fontsize)
    ax.set_ylabel('Career Age', fontsize=fontsize)
    ax.set_title(title, fontsize=fontsize)
    ax.tick_params(axis='x', which='major', direction='in', width=linewidth, size=4*linewidth, labelsize=fontsize)
    ax.tick_params(axis='x', which='minor', direction='in', width=linewidth, size=2*linewidth, labelsize=fontsize)
    ax.tick_params(axis='y', which='major', direction='in', width=linewidth, size=4*linewidth, labelsize=fontsize)
    ax.tick_params(axis='y', which='minor', direction='in', width=linewidth, size=2*linewidth, labelsize=fontsize)
    ax.spines['left'].set_linewidth(linewidth)
    ax.spines['right'].set_linewidth(linewidth)
    ax.spines['bottom'].set_linewidth(linewidth)
    ax.spines['top'].set_linewidth(linewidth)
    plt.gcf().text(0., 0.9, letter, fontsize=fontsize*2)
    plt.subplots_adjust(left=0.2, right=0.9, bottom=0.2, top=0.9)
    plt.plot()
    fig.savefig(f'../results/fig4/fig4_horiz_ineq_{letter}.pdf')

In [ ]:
# all authors: every-author assignment
cum_cit_pub_agg = counts.groupby(['cohort', 'career_age', 'gender']).agg({'cum_num_pub': list, 'cum_num_cit': list}).reset_index()
mwu_cliff_d_cum_pub = get_cohort_effect_size(cum_cit_pub_agg, 'cum_num_pub')
mwu_cliff_d_cum_cit = get_cohort_effect_size(cum_cit_pub_agg, 'cum_num_cit')
horizonal_inequality(mwu_cliff_d_cum_pub, 'A', 'Productivity')
horizonal_inequality(mwu_cliff_d_cum_cit, 'B', 'Impact')

# all authors: first-author assignment
cum_cit_pub_agg_first = counts_first.groupby(['cohort', 'career_age', 'gender']).agg({'cum_num_pub': list, 'cum_num_cit': list}).reset_index()
mwu_cliff_d_cum_pub_first = get_cohort_effect_size(cum_cit_pub_agg_first, 'cum_num_pub')
mwu_cliff_d_cum_cit_first = get_cohort_effect_size(cum_cit_pub_agg_first, 'cum_num_cit')
horizonal_inequality(mwu_cliff_d_cum_pub_first, 'C', 'Productivity')
horizonal_inequality(mwu_cliff_d_cum_cit_first, 'D', 'Impact')

# dropouts removed: every-author assignment
cum_cit_pub_agg_stayed = counts[counts.dropout == False].groupby(['cohort', 'career_age', 'gender']).agg({'cum_num_pub': list, 'cum_num_cit': list}).reset_index()
mwu_cliff_d_cum_pub_stayed = get_cohort_effect_size(cum_cit_pub_agg_stayed, 'cum_num_pub')
mwu_cliff_d_cum_cit_stayed = get_cohort_effect_size(cum_cit_pub_agg_stayed, 'cum_num_cit')
horizonal_inequality(mwu_cliff_d_cum_pub_stayed, 'E', 'Productivity')
horizonal_inequality(mwu_cliff_d_cum_cit_stayed, 'F', 'Impact')

# dropouts removed: first-author assignment
cum_cit_pub_agg_stayed_first = counts_first[counts_first.dropout == False].groupby(['cohort', 'career_age', 'gender']).agg({'cum_num_pub': list, 'cum_num_cit': list}).reset_index()
mwu_cliff_d_cum_pub_stayed_first = get_cohort_effect_size(cum_cit_pub_agg_stayed_first, 'cum_num_pub')
mwu_cliff_d_cum_cit_stayed_first = get_cohort_effect_size(cum_cit_pub_agg_stayed_first, 'cum_num_cit')
horizonal_inequality(mwu_cliff_d_cum_pub_stayed_first, 'G', 'Productivity')
horizonal_inequality(mwu_cliff_d_cum_cit_stayed_first, 'H', 'Impact')

In [ ]:
linewidth = 2
fontsize = 18
fig = plt.figure(figsize=(1.5, 4))
ax = fig.add_subplot(111)
col_map = plt.get_cmap('bwr')
cb = ColorbarBase(ax, cmap=col_map, orientation='vertical', norm=Normalize(-.17490821707496162, .17490821707496162))
cb.set_label('Cliff\'s $d$', fontsize=fontsize)
cb.outline.set_linewidth(linewidth)
ax.tick_params(labelsize=fontsize, width=linewidth, size=4, direction='in')
plt.subplots_adjust(left=0.1, right=0.3, bottom=0.2, top=0.9)
fig.savefig('../results/fig4/fig4_horiz_ineq_colorbar.pdf')

To interprete these results, compute the number of significant differences:

In [ ]:
print('significant differences')
print(
    '... productivity:', 
    len(mwu_cliff_d_cum_pub[mwu_cliff_d_cum_pub['pvalue'] <= 0.05])
)
print(
    '... impact:', 
    len(mwu_cliff_d_cum_cit[mwu_cliff_d_cum_cit['pvalue'] <= 0.05])
)
print(
    '... productivity (first):', 
    len(mwu_cliff_d_cum_pub_first[mwu_cliff_d_cum_pub_first['pvalue'] <= 0.05])
)
print(
    '... impact (first):', 
    len(mwu_cliff_d_cum_cit_first[mwu_cliff_d_cum_cit_first['pvalue'] <= 0.05])
)
print('of', 15*31, 'observations')

Also compute how strongly horizontal inequality in productivity and impact is correlated:

In [ ]:
mwu_cliff_d_cum = pd.merge(
    left=mwu_cliff_d_cum_pub[mwu_cliff_d_cum_pub['pvalue'] <= 0.05][['cohort', 'career_age', 'cliffd_m_f']], 
    right=mwu_cliff_d_cum_cit[mwu_cliff_d_cum_cit['pvalue'] <= 0.05][['cohort', 'career_age', 'cliffd_m_f']], 
    on=['cohort', 'career_age']
)
mwu_cliff_d_cum.columns = ['cohort', 'career_age', 'cliffd_m_f_pub', 'cliffd_m_f_cit']

mwu_cliff_d_cum_first = pd.merge(
    left=mwu_cliff_d_cum_pub_first[mwu_cliff_d_cum_pub_first['pvalue'] <= 0.05][['cohort', 'career_age', 'cliffd_m_f']], 
    right=mwu_cliff_d_cum_cit_first[mwu_cliff_d_cum_cit_first['pvalue'] <= 0.05][['cohort', 'career_age', 'cliffd_m_f']], 
    on=['cohort', 'career_age']
)
mwu_cliff_d_cum_first.columns = ['cohort', 'career_age', 'cliffd_m_f_pub', 'cliffd_m_f_cit']

Set `df = mwu_cliff_d_cum` and `df = mwu_cliff_d_cum_first` to get results for every-author assignment and first-author assignment, respectively:

In [ ]:
df = mwu_cliff_d_cum

matrix_r = []
matrix_p = []
for i in range(4):
    row_r = []
    row_p = []
    for j in range(4):
        r, p = sp.stats.pearsonr(
            df.iloc[:, i], 
            df.iloc[:, j]
        )
        row_r.append(r)
        row_p.append(p)
    matrix_r.append(row_r)
    matrix_p.append(row_p)
df_r = pd.DataFrame(matrix_r, index=df.columns, columns=df.columns)
df_p = pd.DataFrame(matrix_p, index=df.columns, columns=df.columns)
df_r = df_r.style.set_caption('Pearson correlation coefficient')
df_p = df_p.style.set_caption('Significance')
display(df_r)
display(df_p)